In [24]:
import random
import numpy as np
import pandas as pd

# first choice, play creature, spell, land or pass 
# (automatically passes when out of mana)
count_zero = 0
count_one = 0
count_seven = 0
count_eight = 0
for x in range(100000):
    choice = random.choice([0,1,7,8])
    if choice == 0:
        count_zero += 1
    elif choice == 1:
        count_one += 1
    elif choice == 7:
        count_seven += 1
    else:
        count_eight += 1
print(count_zero)
print(count_one)
print(count_seven)
print(count_eight)


24811
24948
25170
25071
